In [2]:
import pandas as pd

# Load the datasets
train_logs = pd.read_csv('train_logs_aggregated.csv')
train_scores = pd.read_csv('train_scores.csv')
combined_data = pd.merge(train_logs, train_scores, on='id')
combined_data.head()

test_logs = pd.read_csv('test_logs_aggregated.csv')
test_scores = pd.read_csv('test')

,id,time_worked,avg_action_time,Input,Move From To,Nonproduction,Paste,Remove/Cut,Replace,count_letters,count_special_chars,final_word_count,score
0,001519c8,1797443,116.246774,2010,3,120,0,417,7,2016,1489,256,3.5
1,0022f953,1758346,112.221271,1938,0,254,1,260,1,1705,2538,323,3.5
2,0042269b,1767228,101.837766,3515,0,175,0,439,7,3675,2210,404,6.0
3,0059420b,1363074,121.848329,1304,0,99,1,151,1,1168,1108,206,2.0
4,0075873a,1584002,123.943896,1942,0,72,0,517,0,1964,1071,252,4.0


In [3]:
# Re-importing necessary libraries and reloading the data as the code execution state was reset
from sklearn.linear_model import Lasso
from sklearn.kernel_ridge import KernelRidge
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split

# Splitting the dataset into features (X) and target (y) for regression
X = combined_data.drop(['id', 'score'], axis=1)
y_regression = combined_data['score']

# Splitting the datasets into training and testing sets for regression
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X, y_regression, test_size=0.3, random_state=42)

# Since Lasso and Kernel Ridge regression are sensitive to the scale of input, we scale the features
scaler = StandardScaler()

# Lasso Regression
lasso = Lasso(random_state=42)

# Kernel Ridge Regression
kernel_ridge = KernelRidge()

# Creating pipelines for both models
pipeline_lasso = Pipeline([('scaler', scaler), ('lasso', lasso)])
pipeline_kernel_ridge = Pipeline([('scaler', scaler), ('kernel_ridge', kernel_ridge)])

# Grid search for hyperparameter tuning
# Note: The range of parameters is chosen for demonstration and should be adjusted based on the dataset.
param_grid_lasso = {'lasso__alpha': [0.001, 0.01, 0.1, 1, 10]}
param_grid_kernel_ridge = {'kernel_ridge__alpha': [0.001, 0.01, 0.1, 1, 10],
                           'kernel_ridge__kernel': ['linear', 'rbf', 'poly']}

grid_search_lasso = GridSearchCV(pipeline_lasso, param_grid_lasso, cv=5, scoring='neg_mean_squared_error')
grid_search_kernel_ridge = GridSearchCV(pipeline_kernel_ridge, param_grid_kernel_ridge, cv=5, scoring='neg_mean_squared_error')

# Fit models to the regression training set
grid_search_lasso.fit(X_train_reg, y_train_reg)
grid_search_kernel_ridge.fit(X_train_reg, y_train_reg)

# Best parameters and scores
best_params_lasso = grid_search_lasso.best_params_
best_score_lasso = -grid_search_lasso.best_score_

best_params_kernel_ridge = grid_search_kernel_ridge.best_params_
best_score_kernel_ridge = -grid_search_kernel_ridge.best_score_

best_params_lasso, best_score_lasso, best_params_kernel_ridge, best_score_kernel_ridge


/Users/justintang/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4.245849730596433, tolerance: 0.11544821428571429
  model = cd_fast.enet_coordinate_descent(
/Users/justintang/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4.2485141881270465, tolerance: 0.1164315551537071
  model = cd_fast.enet_coordinate_descent(
/Users/justintang/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 21.38548346062794, tolerance: 0.11962531645569621
  model = cd_fast.enet_coordinate_descent(
/Users/justintang/opt/anaconda3/lib/python3.9/site-p

({'lasso__alpha': 0.001},
 0.6069242748281806,
 {'kernel_ridge__alpha': 0.1, 'kernel_ridge__kernel': 'rbf'},
 0.7679625280757953)

In [4]:
#tuning kernel ridge regression

# Splitting the dataset into features (X) and target (y) for regression
X = combined_data.drop(['id', 'score'], axis=1)
y_regression = combined_data['score']

# Splitting the datasets into training and testing sets for regression
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X, y_regression, test_size=0.3, random_state=42)

# Kernel Ridge Regression with Grid Search for hyperparameter tuning
# Parameters for GridSearchCV
param_grid = {
    'alpha': [0.001, 0.01, 0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly']
}

# Creating a new Kernel Ridge object
kernel_ridge_reg_tuned = KernelRidge()

# Grid Search with 5-fold cross-validation
grid_search = GridSearchCV(kernel_ridge_reg_tuned, param_grid, cv=5, scoring='neg_mean_squared_error')

# Fit the grid search to the data
grid_search.fit(X_train_reg, y_train_reg)

# Best parameters and best score
best_params = grid_search.best_params_
best_score = -grid_search.best_score_  # Negative MSE to MSE

best_params, best_score

/Users/justintang/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:190: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
/Users/justintang/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:190: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
/Users/justintang/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:190: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
/Users/justintang/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:190: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual p

({'alpha': 0.01, 'kernel': 'linear'}, 0.6749873364702846)

In [7]:
# tuning lasso
# Parameters for GridSearchCV
param_grid_lasso = {
    'alpha': [0.001, 0.01, 0.1, 1, 10]
}

# Creating a new Lasso object
lasso_reg_tuned = Lasso(random_state=42)

# Grid Search with 5-fold cross-validation
grid_search_lasso = GridSearchCV(lasso_reg_tuned, param_grid_lasso, cv=5, scoring='neg_mean_squared_error')

# Fit the grid search to the data
grid_search_lasso.fit(X_train_reg, y_train_reg)

# Best parameters and best score
best_params_lasso = grid_search_lasso.best_params_
best_score_lasso = -grid_search_lasso.best_score_  # Negative MSE to MSE

best_params_lasso, best_score_lasso

/Users/justintang/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 315.54802245124466, tolerance: 0.11544821428571429
  model = cd_fast.enet_coordinate_descent(
/Users/justintang/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 312.45632483673893, tolerance: 0.1164315551537071
  model = cd_fast.enet_coordinate_descent(
/Users/justintang/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 333.63769963526977, tolerance: 0.11962531645569621
  model = cd_fast.enet_coordinate_descent(
/Users/justintang/opt/anaconda3/lib/python3.9/site

({'alpha': 0.01}, 0.6042874201973221)

In [9]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Re-initialize Random Forest Regressor
rf_reg = RandomForestRegressor(random_state=42)

# Fitting the Random Forest Regressor to the training data
rf_reg.fit(X_train_reg, y_train_reg)

# Predicting on the test set
rf_predictions = rf_reg.predict(X_test_reg)

# Evaluating the model
mse_rf = mean_squared_error(y_test_reg, rf_predictions)
r2_rf = r2_score(y_test_reg, rf_predictions)

mse_rf, r2_rf

(0.5730604974704889, 0.4682130332122505)

In [10]:
#gradient boosting regressor
from sklearn.ensemble import GradientBoostingRegressor

# Initialize Gradient Boosting Regressor
gb_reg = GradientBoostingRegressor(random_state=42)

# Fitting the Gradient Boosting Regressor to the training data
gb_reg.fit(X_train_reg, y_train_reg)

# Predicting on the test set
gb_predictions = gb_reg.predict(X_test_reg)

# Evaluating the model
mse_gb = mean_squared_error(y_test_reg, gb_predictions)
r2_gb = r2_score(y_test_reg, gb_predictions)

mse_gb, r2_gb


(0.5674386105040331, 0.4734300150679386)

In [11]:
# tuning gradient booster
# Define a grid of hyperparameters to tune
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 0.9, 1.0]
}

# Initialize the GridSearchCV with the Gradient Boosting Regressor
grid_search = GridSearchCV(estimator=gb_reg, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Fitting GridSearchCV to the training data
grid_search.fit(X_train_reg, y_train_reg)

# Best parameters and best score
best_params = grid_search.best_params_
best_score = -grid_search.best_score_

best_params, best_score

({'learning_rate': 0.01,
  'max_depth': 3,
  'n_estimators': 300,
  'subsample': 0.8},
 0.510584179906471)

In [13]:
# random forest classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Convert the 'score' to discrete values for classification
y_classification = y_regression.apply(lambda x: int(x))

# Splitting the dataset into training and testing sets for classification
X_train_cls, X_test_cls, y_train_cls, y_test_cls = train_test_split(X, y_classification, test_size=0.3, random_state=42)

# Initialize Random Forest Classifier
rf_cls = RandomForestClassifier(random_state=42)

# Fitting the Random Forest Classifier to the training data
rf_cls.fit(X_train_cls, y_train_cls)

# Predicting on the test set
rf_cls_predictions = rf_cls.predict(X_test_cls)

# Evaluating the model
accuracy_rf_cls = accuracy_score(y_test_cls, rf_cls_predictions)
report_rf_cls = classification_report(y_test_cls, rf_cls_predictions)

print(accuracy_rf_cls)
print(report_rf_cls)

0.49409780775716694
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.50      0.14      0.22        29
           2       0.29      0.24      0.26        58
           3       0.52      0.58      0.55       200
           4       0.53      0.64      0.58       220
           5       0.39      0.26      0.31        73
           6       0.00      0.00      0.00        12

    accuracy                           0.49       593
   macro avg       0.32      0.27      0.27       593
weighted avg       0.47      0.49      0.47       593



/Users/justintang/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/justintang/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/justintang/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

# Define your parameter grid and initialize RandomizedSearchCV
param_grid = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_features': ['auto', 'sqrt'],
    'max_depth': [10, 20, 30, 40, 50, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}
random_search = RandomizedSearchCV(RandomForestClassifier(random_state=42), 
                                   param_grid, n_iter=100, cv=5, verbose=2, 
                                   random_state=42, n_jobs=-1)

# Fit the model
random_search.fit(X_train_cls, y_train_cls)

# Best parameters and score
best_params = random_search.best_params_
best_score = random_search.best_score_
print(best_params)
print(best_score)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


/Users/justintang/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


[CV] END bootstrap=False, max_depth=30, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=200; total time=   1.2s
[CV] END bootstrap=False, max_depth=30, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=200; total time=   1.2s
[CV] END bootstrap=False, max_depth=30, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=200; total time=   1.2s
[CV] END bootstrap=False, max_depth=30, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=200; total time=   1.2s
[CV] END bootstrap=False, max_depth=30, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=200; total time=   1.2s
[CV] END bootstrap=True, max_depth=50, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=400; total time=   2.0s
[CV] END bootstrap=True, max_depth=50, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=400; total time=   2.1s
[CV] END bootstrap=True, max_depth=50,

In [14]:
# SVM c classifier (took to long to tune so I'm giving up on this)
from sklearn.svm import SVC

# Initialize Support Vector Classifier
svc = SVC(random_state=42)

# Fitting the SVC to the training data
svc.fit(X_train_cls, y_train_cls)

# Predicting on the test set
svc_predictions = svc.predict(X_test_cls)

# Evaluating the model
accuracy_svc = accuracy_score(y_test_cls, svc_predictions)
report_svc = classification_report(y_test_cls, svc_predictions)

print(accuracy_svc)
print(report_svc)

0.36930860033726814
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.00      0.00      0.00        29
           2       0.00      0.00      0.00        58
           3       0.25      0.01      0.01       200
           4       0.37      0.99      0.54       220
           5       0.00      0.00      0.00        73
           6       0.00      0.00      0.00        12

    accuracy                           0.37       593
   macro avg       0.09      0.14      0.08       593
weighted avg       0.22      0.37      0.20       593



/Users/justintang/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/justintang/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/justintang/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
